<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLN-291121_Regres_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
import numpy as np
# import pandas as pd
from sklearn import preprocessing

# StandardScaler

In [172]:
x=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
# x=np.array(x)
scaler = preprocessing.StandardScaler().fit(x)
print(scaler.mean_, scaler.scale_)
x_scaled = scaler.transform(x)
print (x_scaled)
x_back=scaler.inverse_transform(x_scaled)
print (x_back)
scaler.inverse_transform([[0,0,0]])
# help(scaler)

[ 0.          0.33333333 -0.16666667] [1.15470054 0.74535599 0.68718427]
[[ 1.73205081  0.89442719  0.24253563]
 [-0.8660254  -0.4472136   1.69774938]
 [ 0.         -0.4472136   0.24253563]
 [ 0.8660254   0.89442719  0.24253563]
 [-0.8660254  -1.78885438 -1.21267813]
 [-0.8660254   0.89442719 -1.21267813]]
[[ 2.  1.  0.]
 [-1.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  1.  0.]
 [-1. -1. -1.]
 [-1.  1. -1.]]


array([[ 0.        ,  0.33333333, -0.16666667]])

# Single Neuron

In [173]:
class Neuron:
    """A single neuron with an activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0.
          activ:  The activation function: linear (default), relu, sigmoid."""

    def __init__(self, inputs, bias = 1.0, activ = 'linear'):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias) and random initial weights.""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias
        self.activ = activ

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        if self.activ == 'linear':
          return sum
        if self.activ == 'sigmoid':
          return self.sigmoid(sum)
        if self.activ == 'relu':
          return self.relu(sum)
        if self.activ == 'mrelu':
          return self.mrelu(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight."""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init, dtype='double')

    def set_activ(self, activ, param=0):
        """Overrides the 'linear' activation function."""
        self.activ = activ
        self.param = param

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))
    
    def relu(self, x):
        # return the output of the relu function applied to x
        if x >= 0:
          return x
        return 0

    def mrelu(self, x):
        # return the output of the modified relu function applied to x
        if x >= 0:
          return x
        return self.param*x

In [174]:
neu=Neuron(inputs=2)
neu.set_weights([10,10,-15]) 
neu.set_activ('mrelu', param=0.01)
neu.run([1,0])

-0.05

## AND gate

In [175]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-15]) #AND gate

print("AND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


AND Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


## OR gate

In [176]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-5]) #OR gate

print("OR Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


OR Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [177]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([-10,-10,15]) #NAND gate

print("NAND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

NAND Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer neuron

In [178]:
class MultiLayerNeuron:     
    """A multilayer neuron class that uses the Neuron class above.
       Builds a list of neurons with the specific activation function.
       The activation function may be modified later using the set_activ method.
       For example: mln.network[layer][neuron].set_activ('linear').
       Attributes:
          layers:  A list with the number of neurons per layer. Including the input (first) and the output (last) layers.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate.
          activ:   The activation function: linear (default), relu, sigmoid."""

    def __init__(self, layers, bias = 1.0, eta = 0.5, activ='linear'):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)
        self.activ = activ

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    neur=Neuron(inputs = self.layers[i-1], bias = self.bias, activ = self.activ) # 
                    self.network[i].append(neur) # adding j perceptrons
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def set_activ(self, activ, param=0):
        """Set the activation function to every neurons."""
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.network[i][j].set_activ(activ, param) # set_activ for each neuron
        self.param=param
    
    def set_output_activ(self, activ, param=0):
        """Set the activation function to the last (output) neurons."""
        i = len(self.network)-1
        for j in range(self.layers[i]):
            self.network[i][j].set_activ(activ, param) 

    def printWeights(self):
        """Displays a summary of weights and activation functions per layer and neuron."""
        print()
        print('Layer 0 is the Input Layer')
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i,"Neuron",j,":",self.network[i][j].weights,self.network[i][j].activ)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Neuron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp_classif(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the sigmoid function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        self.d[-1] = outputs * (1 - outputs) * (error) # derivative of the SIGMOID function 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error # derivative of the SIGMOID function
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))

    def deriv(self, value, i, j=0):
        if self.network[i][j].activ == 'linear':
          # print ('lin')
          return 1
        if self.network[i][j].activ == 'sigmoid':
          # print ('sig')
          return self.sigmoid(value)*(1-self.sigmoid(value))
        if self.network[i][j].activ == 'relu':
          if value > 0:
            # print ('re>')
            return 1
          else:
            # print ('re<')
            return 0
        if self.network[i][j].activ == 'mrelu':
          if value > 0:
            return 1
          else:
            return self.param

    def bp_regres(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative according each neuron's activation function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        for j in range (len(outputs)):
            self.d[-1][j] = self.deriv(outputs[j], len(self.network)-1) * error
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k] 
                self.d[i][h] = self.deriv(self.values[i][h], i, h) * fwd_error
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    # output=sum(weight*value)+bias*bias_weight
                    if k==self.layers[i-1]:
                        # ∂output/∂bias_weight=bias
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        # ∂output/∂weight=value
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] 
                    self.network[i][j].weights[k] += delta # applying the delta rule
        return MSE

## XOR gate=(OR+NAND)+AND

In [179]:
#test code
mln1 = MultiLayerNeuron(layers=[2,2,1])  #mln1
mln1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mln1.set_activ('sigmoid') #linear is by default

mln1.printWeights()
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln1.run([1,1])[0]))


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-10. -10.  15.] sigmoid
Layer 1 Neuron 1 : [ 15.  15. -10.] sigmoid
Layer 2 Neuron 0 : [ 10.  10. -15.] sigmoid

XOR Gate:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098


## Training 

In [180]:
mln2 = MultiLayerNeuron(layers=[2,2,1])
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(2000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

1.141865438648455
1.0875424743085054
1.0420815946273536
0.7269303939464997
0.1472298764118687
0.030917715116783304
0.015773283374714155
0.01035781047136582
0.007639299752604101
0.006021266986977274

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-4.29027381 -4.3091861   6.36587153] sigmoid
Layer 1 Neuron 1 : [-6.03107499 -6.16185698  2.38974534] sigmoid
Layer 2 Neuron 0 : [ 8.4730645  -8.69439911 -3.9472784 ] sigmoid

XOR Gate:
0 0 = 0.0306738465
0 1 = 0.9667414156
1 0 = 0.9663980398
1 1 = 0.0419757056


# Regres

In [181]:
class Regres:
    def __init__(self, layers, bias=1.0, eta=0.1, ):
        self.layers = layers
        self.bias = bias
        self.eta = eta
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias, eta=eta)

    def set_weights(self, w_init):
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ, param=0):
        self.regres_network.set_activ(activ, param=0)
        self.regres_network.set_output_activ('linear')

    def fit(self, X, y, epochs):
        self.epochs=epochs
        history=[]
        self.weight_history=[0.0 for i in range(self.epochs)]
        self.MSE_history=[]
        for i in range(self.epochs):
            weight_epoch=[]
            MSE = 0.0
            for j in range (len(X)):
                MSE +=  self.regres_network.bp_regres(X[j],[y[j]])
            MSE = MSE / len(X)
            self.MSE_history.append(MSE)
            for m in range(1,len(self.regres_network.network)):
                for n in range(self.layers[m]):
                    weight_epoch.append(self.regres_network.network[m][n].weights)
            self.weight_history[i]=weight_epoch
            print ("weight_epoch",weight_epoch)
            # print ('weight_his',self.weight_history)
            # history.append(weight_epoch)
            # print ("history",history)
            # self.weight_history=history
        print ("""Model fitted.
self.weight_history - list of lists of weights propagation
self.MSE_history - list of MSEs propagation""")

    def printWeights(self):
        self.regres_network.printWeights()

    def run(self, x):
        return self.regres_network.run(x)

    def predict(self, x):
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

In [182]:
weight_history=[]
a=[[1,2],[1,3],[4,1]]
b=[]
for m in range(3):
  for n in range(2):
    b.append(a[m][n])
print ("b",b)
weight_history.append(b)
print (weight_history)

b [1, 2, 1, 3, 4, 1]
[[1, 2, 1, 3, 4, 1]]


# Regres_scale

In [183]:
# nuzhno podumat' kak vesa obratno perevesti. mozhet inverse_transform
class Regres_scale:
    def __init__(self, layers, bias=1.0, eta=0.5, ):
        self.layers = layers
        self.bias = bias
        self.eta = eta
        # self.epochs = epochs      
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias, eta=eta)

    def set_weights(self, w_init):
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ):
        self.regres_network.set_activ(activ)
        self.regres_network.set_output_activ('linear')

    def fit(self, x, y, epochs):
        self.scaler = preprocessing.StandardScaler().fit(x)
        x_scaled=self.scaler.transform(x)
        self.epochs=epochs
        for i in range(epochs):
            MSE = 0.0
            for j in range (len(x)):
                MSE +=  self.regres_network.bp_regres(x_scaled[j],[y[j]])
            MSE = MSE / len(x)
            # dobavit' istoriyu kak v Regres

    def printWeights(self):
        self.regres_network.printWeights()

    def run(self, x):
        return self.regres_network.run(x)

    def predict(self, x):
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

    def get_standard_params(self, x):
        x2=np.array(x)
        mean=x2.mean(axis=0)
        stdev=x2.std(axis=0)
        return mean, stdev

# Example 1 -- weights

In [184]:
print ("1 1 =",mln1.run([1,1]))
print (mln1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"

1 1 = [0.00715281]
[-10. -10.  15.]


In [185]:
mln1=MultiLayerNeuron(layers=[2,2,1])
mln1.set_activ('relu')
mln1.printWeights()

mln1.set_output_activ('sigmoid') # setting the output activ func
mln1.printWeights()

mln1.network[1][0].set_activ('linear') # changing specific activ func
mln1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.0762786  0.34623108 0.11998604] relu
Layer 1 Neuron 1 : [ 0.43112262  0.14433069 -0.16652147] relu
Layer 2 Neuron 0 : [-0.33337097  0.02457917  0.92573862] relu


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.0762786  0.34623108 0.11998604] relu
Layer 1 Neuron 1 : [ 0.43112262  0.14433069 -0.16652147] relu
Layer 2 Neuron 0 : [-0.33337097  0.02457917  0.92573862] sigmoid


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.0762786  0.34623108 0.11998604] linear
Layer 1 Neuron 1 : [ 0.43112262  0.14433069 -0.16652147] relu
Layer 2 Neuron 0 : [-0.33337097  0.02457917  0.92573862] sigmoid



# Example 3 -- linear activs, bp_regres. מתכנס למשקלים טוב מאוד 

In [186]:
mln3 = MultiLayerNeuron(layers=[3,1], eta=0.3)
# mln3.set_weights([[[5.,-0.9,2.,0.]]])
mln3.set_weights([[[0,0,0,0]]])
mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(400):
    MSE = 0.0
    MSE += mln3.bp_regres([0.7759,	0.1104,	0.9977,],5.764286995)
    MSE += mln3.bp_regres([0.9692,	0.6961,	0.8483,],5.84646758)
    MSE += mln3.bp_regres([0.0265,	0.399,	0.5375,],0.808633075)
    MSE += mln3.bp_regres([0.7694,	0.5051,	0.2542,],3.850298589)
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()




Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0. 0. 0. 0.] linear


Training Neural Network...

54.560573414122615
9.652321993077698e-13

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.99973369e+00 -9.99625006e-01  1.99991681e+00  3.52253294e-05] linear



# Example 4 -- sigmoid, bp_regres. מקרב רגרסיה עם סיגמויד. עובד טוב

In [187]:
# generating output values
mln4 = MultiLayerNeuron(layers=[3,2,1])
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[-1,2,3]]])
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.1)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')

for i in range(1000):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.  1. -2.  1.] sigmoid
Layer 1 Neuron 1 : [ 1. -2.  0.  1.] sigmoid
Layer 2 Neuron 0 : [-1.  2.  3.] linear



'original values:'

[3.5813200792821274,
 3.731058578630005,
 3.731058578630005,
 3.119202922022118,
 3.880797077977882,
 2.2563920540063265]


MSE= 16.18495519892475
MSE= 0.3146917475609619
MSE= 0.45079560726303486
MSE= 0.48738808198908457
MSE= 0.39849830150254945

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 36.01126864 -71.93065744  36.15850424  -0.75290128] sigmoid
Layer 1 Neuron 1 : [ 3.27086186 -5.95725701  2.71910365 -1.10235425] sigmoid
Layer 2 Neuron 0 : [2.92860239 0.41212283 2.68302883] linear



'predicted values:'

[3.8341123511387956,
 3.78313559257158,
 3.7234782643196827,
 2.6921463756859687,
 3.5754963650032368,
 2.6830297198146122]

# Example 5 -- relu-sigmoid, bp_regres. מתכנס למשקלים לא משהו

In [188]:
mln4 = MultiLayerNeuron(layers=[3,2,1,1]) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[1,0,1]],[[-1,2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1,1], eta=0.3) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')

for i in range(400):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%20 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.  1. -2.  1.] relu
Layer 1 Neuron 1 : [ 1. -2.  0.  1.] relu
Layer 2 Neuron 0 : [1. 0. 1.] sigmoid
Layer 3 Neuron 0 : [-1.  2.] linear



'original values:'

[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]


MSE 0 = 0.5462590412459115
MSE 20 = 0.04846103347268103
MSE 40 = 0.048318724190199676
MSE 60 = 0.04819236692519905
MSE 80 = 0.04807794686758246
MSE 100 = 0.04797295616756813
MSE 120 = 0.04787571054692199
MSE 140 = 0.047785015534762866
MSE 160 = 0.047699989764824584
MSE 180 = 0.047619965750103604
MSE 200 = 0.047544432198117444
MSE 220 = 0.047473000501277486
MSE 240 = 0.0474053866339188
MSE 260 = 0.04734140399679126
MSE 280 = 0.04728096509229237
MSE 300 = 0.04722409129467405
MSE 320 = 0.04717093083738402
MSE 340 = 0.04712178562714422
MSE 360 = 0.04707714756934372
MSE 380 = 0.04703774451535592

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.54950762 -0.68914942 -0.94785424 -0.21644664] relu
Layer 1 Neuron 1 : [ 0.17848226 -0.55960051 -0.33591261 -0.75544821] relu
Layer 2 Neuron 0 : [ 0.30939053 -0.23456568 -0.88470623] sigmoid
Layer 3 Neuron 0 : [-0.1362159   1.06626814] linear



'predicted values:'

[1.0247587789188546,
 1.0264653769671375,
 1.0264653769671375,
 1.0264653769671375,
 1.0184738960660582,
 1.0264653769671375]

# Example 7 -- Regres class. עובד באופן עקרוני
# relu בעייתי

In [189]:
reg1=Regres(layers=[3,2,1,1], eta=0.3)
reg1.set_weights([[[1,1,0,0],[0,0,0,0]],[[0,0,0]],[[0,0]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg1.set_hidden_activ('relu')
reg1.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg1.fit(X,y, epochs=5)
reg1.printWeights()
pred=reg1.predict(X)
print (pred)
print()
# display ('weight_history',reg1.weight_history)
display ('MSE',reg1.MSE_history)

weight_epoch [array([1., 1., 0., 0.]), array([0., 0., 0., 0.]), array([-0.01004363,  0.        ,  0.01054202]), array([0.40626752, 0.81345965])]
weight_epoch [array([ 9.99943510e-01,  1.00005251e+00, -4.50468799e-05,  5.25099344e-05]), array([0., 0., 0., 0.]), array([-0.01639217,  0.        ,  0.00904985]), array([0.40618406, 0.81392458])]
weight_epoch [array([ 9.99878829e-01,  1.00014354e+00, -1.06675990e-04,  2.11560284e-04]), array([0., 0., 0., 0.]), array([-0.02228857,  0.        ,  0.00755239]), array([0.4060741 , 0.81421285])]
weight_epoch [array([ 9.99802753e-01,  1.00026810e+00, -1.83813904e-04,  4.40823369e-04]), array([0., 0., 0., 0.]), array([-0.02773872,  0.        ,  0.00606316]), array([0.405996  , 0.81447959])]
weight_epoch [array([ 9.99713140e-01,  1.00042202e+00, -2.75446741e-04,  7.33518678e-04]), array([0., 0., 0., 0.]), array([-0.03277788,  0.        ,  0.00458208]), array([0.40593827, 0.81473119])]
Model fitted.
self.weight_history - list of lists of weights propag

'MSE'

[0.8936248829442258,
 0.04857883327236249,
 0.0481469881158581,
 0.04777680393002976,
 0.04744442311981101]

# Example -- Regres_scale. צריך חשוב ולהשלים

In [190]:
# nuzhno podumat' kak vesa obratno perevesti. mozhet inverse_transform
reg2=Regres_scale(layers=[3,2,1,1], eta=0.1)
reg2.set_weights([[[1,1,0,0],[0,0,0,0]],[[0,0,0]],[[0,0]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg2.set_hidden_activ('relu')
reg2.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg2.fit(X,y,epochs=200)
reg2.printWeights()
pred=reg2.predict(X)
pred


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.9517462   1.06201428 -0.06407585  0.06933407] relu
Layer 1 Neuron 1 : [0. 0. 0. 0.] relu
Layer 2 Neuron 0 : [-0.20894407  0.          0.00917428] sigmoid
Layer 3 Neuron 0 : [0.46600036 0.84566849] linear



[1.0081438427807037,
 1.0797374688505368,
 1.078049747612554,
 1.029768336904302,
 1.0797374688505368,
 1.073806571646052]

# Example 6 -- https://mmuratarat.github.io/2020-01-09/backpropagation

In [191]:
# generating output values
mln6 = MultiLayerNeuron(layers=[5,3,1], eta=.5)
mln6.set_activ('sigmoid')
mln6.set_output_activ('linear')
mln6.set_weights([[[.19,.33,.4,.51,.54,.1],
                   [.55,.16,.35,.85,.49,.1],
                   [.76,.97,.7,.85,.57,.1]],
                  [[.1,.03,-.17,.1]]
                  ]) # setting initial weights
mln6.printWeights()

# training
for i in range(100):
    MSE = 0.0
    MSE += mln6.bp_regres([.5,.1,1,0,0],[0.1])
    MSE += mln6.bp_regres([.3,.2,0,1,0],[.6])
    MSE += mln6.bp_regres([.7,.9,0,0,1],[.4])
    MSE += mln6.bp_regres([.8,.1,1,0,0],[.1])
    MSE = MSE / 4
    if(i%10 == 0):
        print ('MSE', i, '=',MSE)

mln6.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.19 0.33 0.4  0.51 0.54 0.1 ] sigmoid
Layer 1 Neuron 1 : [0.55 0.16 0.35 0.85 0.49 0.1 ] sigmoid
Layer 1 Neuron 2 : [0.76 0.97 0.7  0.85 0.57 0.1 ] sigmoid
Layer 2 Neuron 0 : [ 0.1   0.03 -0.17  0.1 ] linear

MSE 0 = 3.097759865123268
MSE 10 = 0.2948017967667316
MSE 20 = 0.11235016703947856
MSE 30 = 0.039031749833149484
MSE 40 = 0.005346702764168406
MSE 50 = 0.00047155150910413527
MSE 60 = 9.822149422722454e-05
MSE 70 = 6.0504726410544755e-05
MSE 80 = 4.67746394940529e-05
MSE 90 = 3.966114337742912e-05

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.92948745 -0.3432972  -0.4785581   0.21811547 -0.0456272  -1.65606983] sigmoid
Layer 1 Neuron 1 : [-1.01311017 -0.23274925 -1.57166574  1.18384536  0.1929425  -1.78487787] sigmoid
Layer 1 Neuron 2 : [-0.09277171 -0.3986799   1.19699659 -0.59778185 -0.68424799 -2.10503325] sigmoid
Layer 2 Neuron 0 : [ 0.25752661  0.97942477 -1.06640139  0.35525422] linear



# Example 8 -- Relu check

In [192]:
reg2=Regres(layers=[1,1,1], eta=0.01)
X=[[0],[1],[2],[3],[4],[5]]
y=[0,1,2,3,4,5]
reg2.printWeights()

reg2.set_hidden_activ('linear')
reg2.fit(X,y,epochs=5)
reg2.printWeights()
pred=reg2.predict(X)
print (pred)
print()
display ('weight_history',reg2.weight_history)
display ('MSE',reg2.MSE_history)


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.95857011  0.14705313] linear
Layer 2 Neuron 0 : [-0.61881202  0.54702895] linear

weight_epoch [array([-1.09073759,  0.11795281]), array([-0.80688341,  0.58963738])]
weight_epoch [array([-1.10012678,  0.13042581]), array([-0.8215007 ,  0.57422415])]
weight_epoch [array([-1.10185797,  0.14478834]), array([-0.82625418,  0.55678058])]
weight_epoch [array([-1.10322915,  0.15839369]), array([-0.83059985,  0.54034959])]
weight_epoch [array([-1.10442429,  0.1712147 ]), array([-0.83474537,  0.52494523])]
Model fitted.
self.weight_history - list of lists of weights propagation
self.MSE_history - list of MSEs propagation

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-1.10442429  0.1712147 ] linear
Layer 2 Neuron 0 : [-0.83474537  0.52494523] linear

[0.38202454890319437, 1.3039376191471266, 2.225850689391059, 3.147763759634991, 4.069676829878923, 4.991589900122855]



'weight_history'

[[array([-1.10442429,  0.1712147 ]), array([-0.83474537,  0.52494523])],
 [array([-1.10442429,  0.1712147 ]), array([-0.83474537,  0.52494523])],
 [array([-1.10442429,  0.1712147 ]), array([-0.83474537,  0.52494523])],
 [array([-1.10442429,  0.1712147 ]), array([-0.83474537,  0.52494523])],
 [array([-1.10442429,  0.1712147 ]), array([-0.83474537,  0.52494523])]]

'MSE'

[1.3725036441082057,
 0.31155315477281936,
 0.2877186109076955,
 0.25240723572643786,
 0.22081958374089586]

# Example 9 -- mrelu check. working

In [193]:
mln4 = MultiLayerNeuron(layers=[1,2,1]) # generating MLN with random weights
mln4.set_activ('mrelu', param=0.1)
# mln4.set_activ('relu')
mln4.set_output_activ('linear')
mln4.set_weights([[[-1.1,1.74],[3.35,-6.44]],[[-1.41,2.05,2.2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([0])[0])
y4.append(mln4.run([1])[0])
y4.append(mln4.run([2])[0])
y4.append(mln4.run([3])[0])
y4.append(mln4.run([4])[0])
y4.append(mln4.run([5])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[1,2,1], eta=0.005) # generating MLN with random weights
mln4.set_activ('mrelu', param=0.1)
# mln4.set_activ('relu')
mln4.set_output_activ('linear')

for i in range(4000):
    MSE = 0.0
    MSE += mln4.bp_regres([0],y4[0])
    MSE += mln4.bp_regres([1],y4[1])
    MSE += mln4.bp_regres([2],y4[2])
    MSE += mln4.bp_regres([3],y4[3])
    MSE += mln4.bp_regres([4],y4[4])
    MSE += mln4.bp_regres([5],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([0])[0])
y4.append(mln4.run([1])[0])
y4.append(mln4.run([2])[0])
y4.append(mln4.run([3])[0])
y4.append(mln4.run([4])[0])
y4.append(mln4.run([5])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-1.1   1.74] mrelu
Layer 1 Neuron 1 : [ 3.35 -6.44] mrelu
Layer 2 Neuron 0 : [-1.41  2.05  2.2 ] linear



'original values:'

[-1.5735999999999999,
 0.6641500000000005,
 2.7978599999999996,
 9.82046,
 16.843059999999998,
 23.865659999999995]


MSE 0 = 612.6643432272605
MSE 200 = 8.333048641430231
MSE 400 = 1.2778662121051203
MSE 600 = 0.754769076233262
MSE 800 = 0.3740768075913423
MSE 1000 = 0.07859955609611667
MSE 1200 = 0.011447242621122754
MSE 1400 = 0.0015037944936402365
MSE 1600 = 0.00019058404755281397
MSE 1800 = 2.3823723939203674e-05
MSE 2000 = 2.96836164684285e-06
MSE 2200 = 3.694008089155483e-07
MSE 2400 = 4.5952768862663815e-08
MSE 2600 = 5.716206371697316e-09
MSE 2800 = 7.109863975432801e-10
MSE 3000 = 8.843112393602159e-11
MSE 3200 = 1.0999570810217392e-11
MSE 3400 = 1.3672108660013314e-12
MSE 3600 = 1.6937700647237486e-13
MSE 3800 = 2.1064422831134935e-14

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-1.10511818  1.10511818] mrelu
Layer 1 Neuron 1 : [ 2.18660129 -3.91617343] mrelu
Layer 2 Neuron 0 : [-1.40346974  3.14071891  1.20735993] linear



'predicted values:'

[-1.57359998203963,
 0.6641499413394282,
 2.797860008094644,
 9.820460003819772,
 16.8430599995449,
 23.865659995270025]

In [194]:
reg2=Regres(layers=[1,2,1], eta=0.0001)
X=[[1],[2],[3],[4],[5]]
y=[1.,4.,9.,16.,25.]
# reg2.printWeights()

reg2.set_hidden_activ('mrelu', param=0.1)
reg2.fit(X,y,epochs=500)
reg2.printWeights()
pred=reg2.predict(X)
print (pred)
print()
# display ('weight_history',reg2.weight_history)
display ('MSE',np.array(reg2.MSE_history).min())

weight_epoch [array([-0.68412759, -0.0836259 ]), array([0.45478376, 0.46399496]), array([-0.96154489, -0.50658509, -0.56748191])]
weight_epoch [array([-0.68412759, -0.0836259 ]), array([0.42999403, 0.45777451]), array([-0.96154489, -0.47854005, -0.55501007])]
weight_epoch [array([-0.68412759, -0.0836259 ]), array([0.40674087, 0.45194602]), array([-0.96154489, -0.4519745 , -0.54263845])]
weight_epoch [array([-0.68412759, -0.0836259 ]), array([0.38492004, 0.44648213]), array([-0.96154489, -0.42677482, -0.53035796])]
weight_epoch [array([-0.68412759, -0.0836259 ]), array([0.36443806, 0.44135839]), array([-0.96154489, -0.40283839, -0.51816053])]
weight_epoch [array([-0.68412759, -0.0836259 ]), array([0.34521084, 0.43655283]), array([-0.96154489, -0.38007208, -0.506039  ])]
weight_epoch [array([-0.68412759, -0.0836259 ]), array([0.32716246, 0.43204566]), array([-0.96154489, -0.35839097, -0.493987  ])]
weight_epoch [array([-0.68412759, -0.0836259 ]), array([0.31022417, 0.42781902]), array([-

'MSE'

42.48581385228632